In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#  Create a test graph
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [18]:
number_of_inputs = 28 * 28
n_hidden1 = 1000
n_hidden2 = 500
n_hidden3 = 100
n_hidden4 = 50
n_output = 10

def neuron_layer(X, n_neurons, name):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        # truncated normal distributions limit the size of the weights, speeding up the training time.
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        return tf.nn.relu(z)

with tf.device("/gpu:0"):
    x = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="input")
    y = tf.placeholder(tf.int64, shape=(None), name="y")

    with tf.name_scope("dnn"):
        hidden1_output = neuron_layer(x, n_hidden1, "hidden1")
        hidden2_output = neuron_layer(hidden1_output, n_hidden2, "hidden2")
        hidden3_output = neuron_layer(hidden2_output, n_hidden3, "hidden3")
        hidden4_output = neuron_layer(hidden3_output, n_hidden4, "hidden4")
        logits = neuron_layer(hidden4_output, n_output, "output")

    with tf.name_scope("loss"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(cross_entropy, name="loss")

    with tf.name_scope("training"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    k = 1
    correctness = tf.nn.in_top_k(logits, y, k)
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32)) * 100

In [19]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/mnist_model.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge([loss_summary, accuracy_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [20]:
epochs = 10
batch_size = 100
n_batches = int(np.ceil(mnist.train.num_examples // batch_size))

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    #saver.restore(sess, interim_checkpoint_path)
    
    for epoch in range(epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            if batch_index % 10 == 0:
                summary_str = summary_op.eval(feed_dict={x: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            t, l, a = sess.run([training_op, loss, accuracy], feed_dict={x: X_batch, y: y_batch})
            
            if batch_index % 10000: print("loss:", l, "accuracy:", a)
        if epoch % 1 == 0:
            save_path = saver.save(sess, interim_checkpoint_path)
            test_acc = accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
            
            print(">>>>>>>>>> test dataset accuracy:", test_acc)

    save_path = saver.save(sess, "./checkpoints/mnist_model_final.ckpt")

loss: 2.36651 accuracy: 15.0
loss: 2.29901 accuracy: 16.0
loss: 2.15483 accuracy: 25.0
loss: 2.11218 accuracy: 31.0
loss: 1.99116 accuracy: 34.0
loss: 2.00267 accuracy: 35.0
loss: 1.94445 accuracy: 39.0
loss: 1.87665 accuracy: 39.0
loss: 2.05443 accuracy: 28.0
loss: 1.81381 accuracy: 41.0
loss: 2.19886 accuracy: 33.0
loss: 2.63245 accuracy: 24.0
loss: 3.2966 accuracy: 11.0
loss: 2.10924 accuracy: 21.0
loss: 2.10728 accuracy: 34.0
loss: 2.16004 accuracy: 17.0
loss: 1.89746 accuracy: 42.0
loss: 1.50495 accuracy: 59.0
loss: 1.53296 accuracy: 57.0
loss: 1.53048 accuracy: 56.0
loss: 1.33637 accuracy: 63.0
loss: 1.11283 accuracy: 66.0
loss: 1.39557 accuracy: 54.0
loss: 1.41595 accuracy: 55.0
loss: 1.07627 accuracy: 66.0
loss: 1.0198 accuracy: 69.0
loss: 0.994305 accuracy: 72.0
loss: 1.03293 accuracy: 68.0
loss: 0.962346 accuracy: 70.0
loss: 0.676946 accuracy: 83.0
loss: 0.957636 accuracy: 75.0
loss: 0.676229 accuracy: 80.0
loss: 0.866485 accuracy: 75.0
loss: 1.01577 accuracy: 67.0
loss: 1.16

loss: 0.143068 accuracy: 94.0
loss: 0.174443 accuracy: 95.0
loss: 0.105212 accuracy: 95.0
loss: 0.0861856 accuracy: 98.0
loss: 0.133619 accuracy: 96.0
loss: 0.238014 accuracy: 95.0
loss: 0.163118 accuracy: 95.0
loss: 0.132588 accuracy: 94.0
loss: 0.163699 accuracy: 96.0
loss: 0.227814 accuracy: 94.0
loss: 0.314801 accuracy: 90.0
loss: 0.209582 accuracy: 94.0
loss: 0.105339 accuracy: 96.0
loss: 0.182652 accuracy: 95.0
loss: 0.102623 accuracy: 96.0
loss: 0.18092 accuracy: 96.0
loss: 0.12752 accuracy: 97.0
loss: 0.156601 accuracy: 96.0
loss: 0.102581 accuracy: 98.0
loss: 0.262397 accuracy: 94.0
loss: 0.108029 accuracy: 97.0
loss: 0.170005 accuracy: 94.0
loss: 0.187631 accuracy: 92.0
loss: 0.0648149 accuracy: 98.0
loss: 0.199785 accuracy: 92.0
loss: 0.148929 accuracy: 95.0
loss: 0.242221 accuracy: 93.0
loss: 0.142532 accuracy: 95.0
loss: 0.257073 accuracy: 93.0
loss: 0.0982527 accuracy: 98.0
loss: 0.277202 accuracy: 93.0
loss: 0.182802 accuracy: 97.0
loss: 0.147772 accuracy: 94.0
loss: 0.0

loss: 0.0983948 accuracy: 98.0
loss: 0.101465 accuracy: 99.0
loss: 0.0955366 accuracy: 95.0
loss: 0.0979097 accuracy: 97.0
loss: 0.0818639 accuracy: 96.0
loss: 0.0663029 accuracy: 98.0
loss: 0.0523059 accuracy: 98.0
loss: 0.0517511 accuracy: 98.0
loss: 0.116945 accuracy: 97.0
loss: 0.135647 accuracy: 96.0
loss: 0.117605 accuracy: 95.0
loss: 0.0464652 accuracy: 99.0
loss: 0.206183 accuracy: 95.0
loss: 0.178332 accuracy: 93.0
loss: 0.096166 accuracy: 95.0
loss: 0.121548 accuracy: 96.0
loss: 0.0372559 accuracy: 100.0
loss: 0.138542 accuracy: 96.0
loss: 0.112176 accuracy: 97.0
loss: 0.140771 accuracy: 97.0
loss: 0.101898 accuracy: 97.0
loss: 0.195724 accuracy: 95.0
loss: 0.130778 accuracy: 96.0
loss: 0.0979172 accuracy: 98.0
loss: 0.154416 accuracy: 96.0
loss: 0.0852567 accuracy: 98.0
loss: 0.092098 accuracy: 98.0
loss: 0.0882537 accuracy: 99.0
loss: 0.0843941 accuracy: 98.0
loss: 0.087831 accuracy: 98.0
loss: 0.189462 accuracy: 96.0
loss: 0.0489697 accuracy: 99.0
loss: 0.0778981 accuracy:

loss: 0.0615506 accuracy: 97.0
loss: 0.0426345 accuracy: 98.0
loss: 0.0567875 accuracy: 96.0
loss: 0.0672125 accuracy: 97.0
loss: 0.0976917 accuracy: 97.0
loss: 0.0854013 accuracy: 96.0
loss: 0.103681 accuracy: 96.0
loss: 0.0812463 accuracy: 95.0
loss: 0.0911084 accuracy: 96.0
loss: 0.192348 accuracy: 93.0
loss: 0.122805 accuracy: 96.0
loss: 0.211828 accuracy: 97.0
loss: 0.0910921 accuracy: 96.0
loss: 0.0199493 accuracy: 100.0
loss: 0.0796417 accuracy: 98.0
loss: 0.150319 accuracy: 96.0
loss: 0.0800245 accuracy: 98.0
loss: 0.160095 accuracy: 96.0
loss: 0.0490314 accuracy: 99.0
loss: 0.108447 accuracy: 94.0
loss: 0.0147867 accuracy: 100.0
loss: 0.0845139 accuracy: 97.0
loss: 0.254245 accuracy: 94.0
loss: 0.0636924 accuracy: 99.0
loss: 0.0561411 accuracy: 99.0
loss: 0.0234214 accuracy: 100.0
loss: 0.0854868 accuracy: 98.0
loss: 0.0817208 accuracy: 96.0
loss: 0.0702447 accuracy: 98.0
loss: 0.118431 accuracy: 96.0
loss: 0.0972277 accuracy: 97.0
loss: 0.173837 accuracy: 96.0
loss: 0.0322145

loss: 0.0321124 accuracy: 99.0
loss: 0.0904256 accuracy: 98.0
loss: 0.0204486 accuracy: 100.0
loss: 0.0530792 accuracy: 98.0
loss: 0.094785 accuracy: 97.0
loss: 0.0412973 accuracy: 99.0
loss: 0.0897626 accuracy: 98.0
loss: 0.0255332 accuracy: 100.0
loss: 0.0780249 accuracy: 96.0
loss: 0.0755791 accuracy: 96.0
loss: 0.0238643 accuracy: 100.0
loss: 0.0841488 accuracy: 98.0
loss: 0.025342 accuracy: 100.0
loss: 0.123984 accuracy: 97.0
loss: 0.0470971 accuracy: 99.0
loss: 0.0941565 accuracy: 97.0
loss: 0.0354718 accuracy: 98.0
loss: 0.0496177 accuracy: 99.0
loss: 0.0275361 accuracy: 99.0
loss: 0.0573838 accuracy: 98.0
loss: 0.0834087 accuracy: 99.0
loss: 0.0790734 accuracy: 97.0
loss: 0.0275195 accuracy: 99.0
loss: 0.0440719 accuracy: 99.0
loss: 0.122027 accuracy: 96.0
loss: 0.0600223 accuracy: 99.0
loss: 0.071375 accuracy: 98.0
loss: 0.0421621 accuracy: 98.0
loss: 0.0571945 accuracy: 98.0
loss: 0.106216 accuracy: 96.0
loss: 0.0763776 accuracy: 97.0
loss: 0.0350131 accuracy: 99.0
loss: 0.12

...

loss: 0.00373367 accuracy: 100.0
loss: 0.00141464 accuracy: 100.0
loss: 0.00032573 accuracy: 100.0
loss: 0.00157615 accuracy: 100.0
loss: 0.00152779 accuracy: 100.0
loss: 0.00645663 accuracy: 100.0
loss: 0.00111038 accuracy: 100.0
loss: 0.00718164 accuracy: 100.0
loss: 0.00294254 accuracy: 100.0
loss: 0.00569331 accuracy: 100.0
loss: 0.00120978 accuracy: 100.0
loss: 0.00481159 accuracy: 100.0
loss: 0.00980274 accuracy: 100.0
loss: 0.000817407 accuracy: 100.0
loss: 0.00924475 accuracy: 100.0
loss: 0.00347821 accuracy: 100.0
loss: 0.0106965 accuracy: 99.0
loss: 0.0158682 accuracy: 99.0
loss: 0.00396815 accuracy: 100.0
loss: 0.000976906 accuracy: 100.0
loss: 0.00127462 accuracy: 100.0
loss: 0.00537688 accuracy: 100.0
loss: 0.00275509 accuracy: 100.0
loss: 0.00819595 accuracy: 100.0
loss: 0.00593324 accuracy: 100.0
loss: 0.030322 accuracy: 99.0
loss: 0.00828513 accuracy: 100.0
loss: 0.00286803 accuracy: 100.0
loss: 0.00796134 accuracy: 100.0
loss: 0.00934699 accuracy: 100.0
loss: 0.0329569

loss: 0.00167651 accuracy: 100.0
loss: 0.00147694 accuracy: 100.0
loss: 0.00524344 accuracy: 100.0
loss: 0.000835472 accuracy: 100.0
loss: 0.00797888 accuracy: 100.0
loss: 0.000612926 accuracy: 100.0
loss: 0.000572985 accuracy: 100.0
loss: 0.00528231 accuracy: 100.0
loss: 0.00168807 accuracy: 100.0
loss: 0.000466614 accuracy: 100.0
loss: 0.000973205 accuracy: 100.0
loss: 0.00317956 accuracy: 100.0
loss: 0.00478865 accuracy: 100.0
loss: 0.0037673 accuracy: 100.0
loss: 0.000375706 accuracy: 100.0
loss: 0.0031323 accuracy: 100.0
loss: 0.00892661 accuracy: 100.0
loss: 0.00369967 accuracy: 100.0
loss: 0.00188119 accuracy: 100.0
loss: 0.000634753 accuracy: 100.0
loss: 0.00141561 accuracy: 100.0
loss: 0.00334723 accuracy: 100.0
loss: 0.00338813 accuracy: 100.0
loss: 0.000797698 accuracy: 100.0
loss: 0.0016971 accuracy: 100.0
loss: 0.00177958 accuracy: 100.0
loss: 0.00176983 accuracy: 100.0
loss: 0.00356394 accuracy: 100.0
loss: 0.00181174 accuracy: 100.0
loss: 0.00249829 accuracy: 100.0
loss:

loss: 0.00610393 accuracy: 100.0
loss: 0.000868407 accuracy: 100.0
loss: 0.00142289 accuracy: 100.0
loss: 0.00246698 accuracy: 100.0
loss: 0.00147385 accuracy: 100.0
loss: 0.0634295 accuracy: 99.0
loss: 0.001368 accuracy: 100.0
loss: 0.00305643 accuracy: 100.0
loss: 0.00106648 accuracy: 100.0
loss: 0.00127378 accuracy: 100.0
loss: 0.0202784 accuracy: 99.0
loss: 0.0185308 accuracy: 99.0
loss: 0.00147175 accuracy: 100.0
loss: 0.0013124 accuracy: 100.0
loss: 0.00327725 accuracy: 100.0
loss: 0.00213254 accuracy: 100.0
loss: 0.00257596 accuracy: 100.0
loss: 0.00337693 accuracy: 100.0
loss: 0.00130776 accuracy: 100.0
loss: 0.00164087 accuracy: 100.0
loss: 0.00163533 accuracy: 100.0
loss: 0.0005988 accuracy: 100.0
loss: 0.0020417 accuracy: 100.0
loss: 0.00154414 accuracy: 100.0
loss: 0.000621525 accuracy: 100.0
loss: 0.00129158 accuracy: 100.0
loss: 0.000872761 accuracy: 100.0
loss: 0.00271469 accuracy: 100.0
loss: 0.0203123 accuracy: 99.0
loss: 0.00765281 accuracy: 100.0
>>>>>>>>>> test data

loss: 0.00253369 accuracy: 100.0
loss: 0.000985578 accuracy: 100.0
loss: 0.00129218 accuracy: 100.0
loss: 0.00114973 accuracy: 100.0
loss: 0.00456386 accuracy: 100.0
loss: 0.00142724 accuracy: 100.0
loss: 0.00172734 accuracy: 100.0
loss: 0.000823159 accuracy: 100.0
loss: 0.000690131 accuracy: 100.0
loss: 0.00121108 accuracy: 100.0
loss: 0.000902565 accuracy: 100.0
loss: 0.00095003 accuracy: 100.0
loss: 0.00177972 accuracy: 100.0
loss: 0.00331376 accuracy: 100.0
loss: 0.00333411 accuracy: 100.0
loss: 0.0349408 accuracy: 99.0
loss: 0.000891149 accuracy: 100.0
loss: 0.000524154 accuracy: 100.0
loss: 0.00201996 accuracy: 100.0
loss: 0.000794382 accuracy: 100.0
loss: 0.00161451 accuracy: 100.0
loss: 0.0026703 accuracy: 100.0
loss: 0.00112946 accuracy: 100.0
loss: 0.00154416 accuracy: 100.0
loss: 0.000989909 accuracy: 100.0
loss: 0.00107699 accuracy: 100.0
loss: 0.000715784 accuracy: 100.0
loss: 0.000568577 accuracy: 100.0
loss: 0.00042351 accuracy: 100.0
loss: 0.00093253 accuracy: 100.0
los

loss: 0.00190524 accuracy: 100.0
loss: 0.000725302 accuracy: 100.0
loss: 0.00058021 accuracy: 100.0
loss: 0.00270537 accuracy: 100.0
loss: 0.00296149 accuracy: 100.0
loss: 0.00244899 accuracy: 100.0
loss: 0.00139543 accuracy: 100.0
loss: 0.00131453 accuracy: 100.0
loss: 0.00230661 accuracy: 100.0
loss: 0.000497015 accuracy: 100.0
loss: 0.000357347 accuracy: 100.0
loss: 0.00224727 accuracy: 100.0
loss: 0.000676417 accuracy: 100.0
loss: 0.000573971 accuracy: 100.0
loss: 0.00208787 accuracy: 100.0
loss: 0.000762648 accuracy: 100.0
loss: 0.00421148 accuracy: 100.0
loss: 0.000827569 accuracy: 100.0
loss: 0.00112153 accuracy: 100.0
loss: 0.00044979 accuracy: 100.0
loss: 0.0014759 accuracy: 100.0
loss: 0.00128987 accuracy: 100.0
loss: 0.000898674 accuracy: 100.0
loss: 0.000812084 accuracy: 100.0
loss: 0.00160039 accuracy: 100.0
loss: 0.00189901 accuracy: 100.0
loss: 0.00285435 accuracy: 100.0
loss: 0.00162699 accuracy: 100.0
loss: 0.00114079 accuracy: 100.0
loss: 0.00127264 accuracy: 100.0
lo